In [145]:
import numpy as np

In [146]:
# Parameters
DECK_SIZE = 52
HAND_SIZE = 13
N_TRIALS = 10000

In [147]:
# Calculate largest gap in circular array
def calc_largest_gap(slots, n_slots):
    # Sort the chosen slots
    slots_sorted = np.sort(slots)
    # Calculate gaps between consecutive slots
    gaps = np.diff(slots_sorted, append=slots_sorted[0] + n_slots)
    return np.max(gaps)

In [148]:
# Calculate gaps at both ends of linear array
def calc_end_gaps(slots, n_slots):
    # Sort the chosen slots
    slots_sorted = np.sort(slots)
    # Calculate gaps for both ends
    lower_end_gap = slots_sorted[0]
    upper_end_gap = n_slots - slots_sorted[-1]
    # Calculate gaps at both ends
    return lower_end_gap + upper_end_gap

In [149]:
# Find min and max in wrapped array
def reorder_hand(slots, n_slots):
    # Sort the chosen slots
    slots_sorted = np.sort(slots)
    # Find min and max in wrapped array
    max_gap = 0
    min_idx = None
    max_idx = None
    for i in range(len(slots_sorted) - 1):
        gap = slots_sorted[i+1] - slots_sorted[i]
        if gap > max_gap:
            max_gap = gap
            min_idx = slots_sorted[i+1]
            max_idx = slots_sorted[i]
    
    # Check wrap-around gap
    wrap_around_gap = slots_sorted[0] + n_slots - slots_sorted[-1]
    if wrap_around_gap > max_gap:
        max_gap = wrap_around_gap
        min_idx = slots_sorted[0]
        max_idx = slots_sorted[-1]

    start_idx = np.where(slots_sorted == min_idx)[0][0]
    reordered_slots = np.concatenate((slots_sorted[start_idx:], slots_sorted[:start_idx]))

    return reordered_slots

In [150]:
# Compute cards eliminated in wraparound min-max by round
def wrap_elimination(hand):
    elim_count_by_round = np.zeros(HAND_SIZE - 1)
    min_idx = None
    max_idx = None
    
    for i in range(HAND_SIZE - 1):
        if i % 2 == 0:
            min_idx = hand.pop(0)
            if i == 0:
                elim_count_by_round[i] = 3
            else:
                elim_count_by_round[i] = (min_idx - max_idx) % DECK_SIZE - 1
        else:
            max_idx = hand.pop(-1)
            elim_count_by_round[i] = (min_idx - max_idx) % DECK_SIZE - 1

    return elim_count_by_round

In [151]:
# Compute cards eliminated in linear min-max by round
def linear_elimination(hand):
    elim_count_by_round = np.zeros(HAND_SIZE - 1)
    min_idx = None
    max_idx = None

    for i in range(HAND_SIZE - 1):
        if i % 2 == 0:
            min_idx = hand.pop(0)
            if i == 0:
                elim_count_by_round[i] = min_idx
            else:
                elim_count_by_round[i] = min_idx + DECK_SIZE - max_idx - 1
        else:
            max_idx = hand.pop(-1)
            elim_count_by_round[i] = min_idx + DECK_SIZE - max_idx - 1

    return elim_count_by_round

In [153]:
# Simulate the process
largest_gaps = []
end_gaps = []
cards_eliminated_wrap = np.zeros((N_TRIALS, HAND_SIZE - 1))
cards_eliminated_linear = np.zeros((N_TRIALS, HAND_SIZE - 1))

for i in range(N_TRIALS):
    # Randomly choose 13 unique slots
    chosen_hand = np.random.choice(DECK_SIZE, HAND_SIZE, replace=False)

    # Calculate the largest gap
    gap = calc_largest_gap(chosen_hand, DECK_SIZE)
    ends = calc_end_gaps(chosen_hand, DECK_SIZE)
    largest_gaps.append(gap)
    end_gaps.append(ends)

    # Reorder hand based on wrap-around min-max
    reordered_hand = reorder_hand(chosen_hand, DECK_SIZE)
    sorted_hand = np.sort(chosen_hand)

    # Compute cards eliminated in both cases
    cards_eliminated_wrap[i, :] = wrap_elimination(list(reordered_hand))
    cards_eliminated_linear[i, :] = linear_elimination(list(sorted_hand))

# Calculate the average largest gap
expected_largest_gap = np.mean(largest_gaps)
expected_end_gaps = np.mean(end_gaps)
print(f'Expected largest gap: {expected_largest_gap}, expected end gaps: {expected_end_gaps}, difference: {expected_largest_gap - expected_end_gaps}')

# Calculate the average number of cards eliminated in both cases
expected_cards_eliminated_wrap = np.mean(cards_eliminated_wrap, axis=0)
expected_cards_eliminated_linear = np.mean(cards_eliminated_linear, axis=0)
print(f'Expected cards eliminated in wrap-around min-max: {expected_cards_eliminated_wrap}')
print(f'Expected cards eliminated in linear min-max: {expected_cards_eliminated_linear}')
print(f'Difference: {expected_cards_eliminated_wrap - expected_cards_eliminated_linear}')

Expected largest gap: 11.5792, expected end gaps: 6.5317, difference: 5.0475
Expected cards eliminated in wrap-around min-max: [ 3.     10.5792 13.9495 17.31   20.7345 24.0714 27.467  30.764  34.1629
 37.5327 40.8817 44.2465]
Expected cards eliminated in linear min-max: [ 2.7563  5.5317  9.3188 13.1526 16.9146 20.6829 24.4551 28.2323 32.0295
 35.8474 39.666  43.4068]
Difference: [0.2437 5.0475 4.6307 4.1574 3.8199 3.3885 3.0119 2.5317 2.1334 1.6853
 1.2157 0.8397]


In [ ]:
# Compute expected value / score improvement

In [ ]:
# Graphs